# Fundamentals of Machine Learning (CSCI-UA.473)
## Lab 6: Non-linear Dimensionality Reduction and Autoencoders 

In [ ]:
# Import the necessary packages from sci-kit learn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.cluster import vq # Specifically uesful for K-means clustering
from sklearn import cluster  # Clustering algorithms such as K-means and agglomerative
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE, MDS
from sklearn.metrics import log_loss
from sklearn.metrics.pairwise import pairwise_distances_argmin
from sklearn.utils import resample
from scipy.spatial.distance import squareform #Import squareform, which creates a symmetric matrix from a vector
from palmerpenguins import load_penguins
from matplotlib.colors import ListedColormap


from umap import UMAP
import seaborn as sns
import torchvision
import torch
import torch.nn as nn
from sklearn_som.som import SOM
import scipy.cluster.hierarchy as sch
import time
import math
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '0' ]

## Part 1 : Non Linear Dimensionality Reduction

### Loss measures when dealing with probabilities
When dealing with probabilistic values, we have typically used the log-loss or the cross-entropy loss. These are defined as : <br />
 $L_{\log}(y, p) = -(y \log (p) + (1 - y) \log (1 - p))$

In [ ]:
y_true = [0, 0, 1, 1]
y_pred = [[.9, .1], [.8, .2], [.3, .7], [.01, .99]]
log_loss(y_true, y_pred)

The Kullback-Leibler Divergence score, or KL divergence score, quantifies how much one probability distribution differs from another probability distribution. The intuition for the KL divergence score is that when the probability for an event from P is large, but the probability for the same event in Q is small, there is a large divergence. When the probability from P is small and the probability from Q is large, there is also a large divergence, but not as large as the first case.
It is given by : <br />

$KL(P || Q) =  \sum_{x \in X} P(x) * \log(\frac{P(x)}{Q(x)})$

In [ ]:
# define distributions
events = ['red', 'green', 'blue']
p = [0.10, 0.40, 0.50]
q = [0.80, 0.15, 0.05]

print('P=%.3f Q=%.3f' % (sum(p), sum(q)))
# plot first distribution
plt.subplot(2,1,1).set_title('P')
plt.bar(events, p)
# plot second distribution
plt.subplot(2,1,2).set_title('Q')
plt.bar(events, q)
# show the plot
plt.show()

In [ ]:
# calculate the kl divergence
def kl_divergence(p, q):
    return sum(p[i] * math.log2(p[i]/q[i]) for i in range(len(p)))

In [ ]:
# calculate (P || Q)
kl_pq = kl_divergence(p, q)
print('KL(P || Q): %.3f bits' % kl_pq)
# calculate (Q || P)
kl_qp = kl_divergence(q, p)
print('KL(Q || P): %.3f bits' % kl_qp)

#### Now the algorithms,

In [ ]:
df = datasets.load_digits(as_frame=True)
X,y = df.data, df.target
print(X.shape, y.shape)

In [ ]:
pca = PCA(n_components=2, whiten=True)
pca.fit(X)

In [ ]:
X_pca = pca.transform(X)

In [ ]:
target_ids = range(len(df.target_names))
plt.figure(figsize=(5, 5))
for i, c, label in zip(target_ids, colors, df.target_names):
    plt.scatter(X_pca[y == i, 0], X_pca[y == i, 1], c=c, label=label)
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.legend()
plt.show()

In [ ]:
lda = LinearDiscriminantAnalysis(n_components=2)
X_lda = lda.fit(X, y).transform(X)

In [ ]:
plt.figure(figsize=(5, 5))
for i, c, label in zip(target_ids, colors, df.target_names):
    plt.scatter(X_lda[y==i, 0], X_lda[y==i, 1], c=c, label=label)
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.legend()
plt.show()

### T-distributed Stochastic Neighbor Embedding (t-SNE) using scikit-learn

A statistical method for visualizing high-dimensional data by giving each datapoint a location in either a two or three-dimensional space which can be easily visualized using a scatter plot. It is not recommended for use in analysis such as clustering or outlier detection since it does not necessarily preserve densities or distances well.

In [ ]:
X_embedded = TSNE(n_components=2, perplexity=5, n_jobs=-1).fit_transform(X)
X_embedded.shape

In [ ]:
target_ids = range(len(df.target_names))
plt.figure(figsize=(5, 5))
for i, c, label in zip(target_ids, colors, df.target_names):
    plt.scatter(X_embedded[y == i, 0], X_embedded[y == i, 1], c=c, label=label)

plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.legend()
plt.show()

In [ ]:
# Perplexity is related to the number of nearest neighbors that is used in other manifold learning algorithms
X_embedded = TSNE(n_components=2, perplexity=50, n_jobs=-1, init='pca').fit_transform(X)
X_embedded.shape

In [ ]:
target_ids = range(len(df.target_names))
plt.figure(figsize=(5, 5))
for i, c, label in zip(target_ids, colors, df.target_names):
    plt.scatter(X_embedded[y == i, 0], X_embedded[y == i, 1], c=c, label=label)

plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.legend()
plt.show()

### UMAP : Uniform Manifold Approximation and Projection for Dimension Reduction

In [ ]:
# umap-learn library is designed to be similar to scikit-learn
umap_model = UMAP(n_neighbors=50,min_dist=0.2,random_state=42)

In [ ]:
umap_model.fit(X)
X_umap = umap_model.transform(X)

In [ ]:
target_ids = range(len(df.target_names))
plt.figure(figsize=(5, 5))
for i, c, label in zip(target_ids, colors, df.target_names):
    plt.scatter(X_umap[y == i, 0], X_umap[y == i, 1], c=c, label=label)
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.legend()
plt.show()

### Multi-Dimensional Scaling 
Multidimensional scaling (MDS) seeks a low-dimensional representation of the data in which the distances respect well the distances in the original high-dimensional space.

In general, MDS is a technique used for analyzing similarity or dissimilarity data. It attempts to model similarity or dissimilarity data as distances in a geometric spaces. The data can be ratings of similarity between objects, interaction frequencies of molecules, or trade indices between countries

In [ ]:
mds = MDS(n_components=2, normalized_stress='auto', dissimilarity='euclidean')
X_mds = mds.fit_transform(X)

In [ ]:
target_ids = range(len(df.target_names))
plt.figure(figsize=(5, 5))
for i, c, label in zip(target_ids, colors, df.target_names):
    plt.scatter(X_mds[y == i, 0], X_mds[y == i, 1], c=c, label=label)
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.legend()
plt.show()
# Same as PCA if we use 'euclidean' as distance to minimize

Let's load a custom dataset based on academic disciplines.

In [ ]:
data = np.genfromtxt('fieldsMDS.csv',delimiter=',') # load file as data
print(data.shape)


* Turn the matrix into a vector of pairwise distances
* Remove all nans to leave only pairwise distances remaining
* Create the distance matrix


In [ ]:
dataVec = np.ndarray.flatten(data) 
dataVec = dataVec[~np.isnan(dataVec)] 
D = squareform(dataVec) 
fieldNames = ['Math', 'Physics', 'Chemistry', 'Biology', 'Psych', 'Neuro', 'Econ', 'Sociology', 'DS', 'CS']; #These are the academic fields

#### MDS Hyperparameters
n_components: Looking for a 2D solution, n_init: Number of runs with random initial starting positions, max_iter: Max number of iterations per run, dissimilarity: We already did it, from out distance matrix, 


In [ ]:
mds = MDS(n_components=2, n_init=100, max_iter = 10000, dissimilarity='precomputed', normalized_stress='auto') #Create the mds object
mdsSolution = mds.fit_transform(D) #Actually run the mds


In [ ]:
plt.scatter(mdsSolution[:,0], mdsSolution[:,1], color='blue') #Making the plot, first 2 dimensions
for ii in range(len(mdsSolution)):
    plt.text(mdsSolution[ii,0], mdsSolution[ii,1],fieldNames[ii])

plt.xlabel('MDS axis 1')
plt.ylabel('MDS axis 2')
plt.show()

print(mds.stress_) #How low could we get the stress? [Un-normalized]

## Comparing PCA, t-SNE, MDS and UMAP runtimes

In [ ]:
def data_size_scaling(algorithm, data, sizes=[100, 200, 400 ,800, 1600], n_runs=5):
    result = []
    for size in sizes:
        for run in range(n_runs):
            subsample = resample(data, n_samples=size)
            start_time = time.time()
            algorithm.fit(subsample)
            elapsed_time = time.time() - start_time
            del subsample
            result.append((size, elapsed_time))
    return pd.DataFrame(result, columns=('dataset size', 'runtime (s)'))

In [ ]:
all_algorithms = [
    PCA(),
    UMAP(),
    TSNE(),
    MDS(normalized_stress='auto'),
]
performance_data = {}
for algorithm in all_algorithms:
    
    alg_name = str(algorithm).split('(')[0]
    performance_data[alg_name] = data_size_scaling(algorithm, X, n_runs=5)

    print(f"[{time.asctime(time.localtime())}] Completed {alg_name}")

In [ ]:
for alg_name, perf_data in performance_data.items():
    # print(perf_data['dataset size'], perf_data['runtime (s)'])
    plt.plot(perf_data['dataset size'], perf_data['runtime (s)'], label=alg_name)
plt.xlabel("Number of samples")
plt.ylabel("Time to fit (s)")
plt.legend()
plt.show()

Interactive plot to visualize UMAP results : https://grantcuster.github.io/umap-explorer/

### Density-based spatial clustering of applications with noise (DBSCAN) <a class="anchor" id="third"></a>

DBSCAN requires two parameters: ε (eps) and the minimum number of points required to form a dense region (minPts). It starts with an arbitrary starting point that has not been visited. This point's ε-neighborhood is retrieved, and if it contains sufficiently many points, a cluster is started. Otherwise, the point is labeled as noise. Note that this point might later be found in a sufficiently sized ε-environment of a different point and hence be made part of a cluster.

If a point is found to be a dense part of a cluster, its ε-neighborhood is also part of that cluster. Hence, all points that are found within the ε-neighborhood are added, as is their own ε-neighborhood when they are also dense. This process continues until the density-connected cluster is completely found. Then, a new unvisited point is retrieved and processed, leading to the discovery of a future cluster or noise.

DBSCAN can be used with any distance function (as well as similarity functions or other predicates). The distance function (dist) can therefore be seen as an additional parameter.

The algorithm can be expressed in pseudocode as follows:

```
DBSCAN(DB, distFunc, eps, minPts) {
    C := 0                                                  /* Cluster counter */
    for each point P in database DB {
        if label(P) ≠ undefined then continue               /* Previously processed in inner loop */
        Neighbors N := RangeQuery(DB, distFunc, P, eps)     /* Find neighbors */
        if |N| < minPts then {                              /* Density check */
            label(P) := Noise                               /* Label as Noise */
            continue
        }
        C := C + 1                                          /* next cluster label */
        label(P) := C                                       /* Label initial point */
        SeedSet S := N \ {P}                                /* Neighbors to expand */
        for each point Q in S {                             /* Process every seed point Q */
            if label(Q) = Noise then label(Q) := C          /* Change Noise to border point */
            if label(Q) ≠ undefined then continue           /* Previously processed (e.g., border point) */
            label(Q) := C                                   /* Label neighbor */
            Neighbors N := RangeQuery(DB, distFunc, Q, eps) /* Find neighbors */
            if |N| ≥ minPts then {                          /* Density check (if Q is a core point) */
                S := S ∪ N                                  /* Add new neighbors to seed set */
            }
        }
    }
}
```

where RangeQuery is:

```
RangeQuery(DB, distFunc, Q, eps) {
    Neighbors N := empty list
    for each point P in database DB {                      /* Scan all points in the database */
        if distFunc(Q, P) ≤ eps then {                     /* Compute distance and check epsilon */
            N := N ∪ {P}                                   /* Add to result */
        }
    }
    return N
}

```

In [ ]:
# Read the dataset

crater = pd.read_csv('crater.csv')
assert len (crater) == 1500
assert set (crater.columns) == set (['x_1', 'x_2', 'kmeans_label'])

with open ('crater_counts.txt', 'rt') as fp:
    true_counts = [int (c) for c in fp.read ().split (',')]
    assert sum (crater['kmeans_label'] == 0) == true_counts[0]
    assert sum (crater['kmeans_label'] == 1) == true_counts[1]

def make_scatter_plot (df, x="x_1", y="x_2", hue="label",
                       palette={0: "red", 1: "olive", 2: "blue", 3: "green"},
                       size=5,
                       centers=None):
    if (hue is not None) and (hue in df.columns):
        sns.lmplot (x=x, y=y, hue=hue, data=df, palette=palette,
                    fit_reg=False)
    else:
        sns.lmplot (x=x, y=y, data=df, fit_reg=False)

    if centers is not None:
        plt.scatter (centers[:,0], centers[:,1],
                     marker=u'*', s=500,
                     c=[palette[0]])

def make_scatter_plot2 (df, x="x_1", y="x_2", hue="label", size=5):
    if (hue is not None) and (hue in df.columns):
        sns.lmplot (x=x, y=y, hue=hue, data=df,
                    fit_reg=False)
    else:
        sns.lmplot (x=x, y=y, data=df, fit_reg=False)

make_scatter_plot (crater, hue='kmeans_label')

In [ ]:
## DBSCAN Algorithm
def region_query (p, eps, X):
    # These lines check that the inputs `p` and `X` have
    # the right shape.
    _, dim = X.shape
    assert (p.shape == (dim,)) or (p.shape == (1, dim)) or (p.shape == (dim, 1))
    
    ### BEGIN SOLUTION
    return np.linalg.norm (p - X, axis=1) <= eps
    ### END SOLUTION

def index_set (y):
    """
    Given a boolean vector, this function returns
    the indices of all True elements.
    """
    assert len (y.shape) == 1

    ### BEGIN SOLUTION
    return set (np.where (y)[0])
    ### END SOLUTION

def find_neighbors (eps, X):
    m, d = X.shape
    neighbors = [] # Empty list to start
    ### BEGIN SOLUTION
    for i in range (len (X)):
        n_i = index_set (region_query (X[i, :], eps, X))
        neighbors.append (n_i)
    ### END SOLUTION
    assert len (neighbors) == m
    return neighbors

def find_core_points (s, neighbors):
    assert type (neighbors) is list
    assert all ([type (n) is set for n in neighbors])
    
    core_set = set ()
    ### BEGIN SOLUTION
    for i, n_i in enumerate (neighbors):
        if len (n_i) >= s:
            core_set.add (i)
    ### END SOLUTION
    return core_set

def expand_cluster (p, neighbors, core_set, visited, assignment):
    # Assume the caller performs Steps 1 and 2 of the procedure.
    # That means 'p' must be a core point that is part of a cluster.
    assert (p in core_set) and (p in visited) and (p in assignment)
    
    reachable = set (neighbors[p])  # Step 3
    while reachable:
        q = reachable.pop () # Step 4
        
        # Put your reordered and correctly indented statements here:
        ### BEGIN SOLUTION
        if q not in visited:
            visited.add (q) # Mark q as visited
            if q in core_set:
                reachable |= neighbors[q]
        if q not in assignment:
            assignment[q] = assignment[p]
        ### END SOLUTION
        
    # This procedure does not return anything
    # except via updates to `visited` and
    # `assignment`.
    
def dbscan (eps, s, X):
    clusters = []
    point_to_cluster = {}
    
    neighbors = find_neighbors (eps, X)
    core_set = find_core_points (s, neighbors)
    
    assignment = {}
    next_cluster_id = 0

    visited = set ()
    for i in core_set: # for each core point i
        if i not in visited:
            visited.add (i) # Mark i as visited
            assignment[i] = next_cluster_id
            expand_cluster (i, neighbors, core_set,
                            visited, assignment)
            next_cluster_id += 1

    return assignment, core_set

In [ ]:
## Visualization
X = crater[['x_1', 'x_2']].values
assignment, core_set = dbscan (0.73, 50, X)

print ("Number of core points:", len (core_set))
print ("Number of clusters:", max (assignment.values ()))
print ("Number of unclassified points:", len (X) - len (assignment))

def plot_labels (df, labels):
    df_labeled = df.copy ()
    df_labeled['label'] = labels
    make_scatter_plot2 (df_labeled)

labels = [-1] * len (X)
for i, c in assignment.items ():
    labels[i] = c
plot_labels (crater, labels)


### Hierarchical Clustering <a class="anchor" id="fifth"></a>

Hierarchical clustering involves creating clusters that have a predetermined ordering from top to bottom. For example, all files and folders on the hard disk are organized in a hierarchy. There are two types of hierarchical clustering, Divisive and Agglomerative.

**Divisive method**

In this method we assign all of the observations to a single cluster and then partition the cluster to two least similar clusters. Finally, we proceed recursively on each cluster until there is one cluster for each observation.

**Agglomerative method**
		
In this method we assign each observation to its own cluster. Then, compute the similarity (e.g., distance) between each of the clusters and join the two most similar clusters. Finally, repeat steps 2 and 3 until there is only a single cluster left.

#### Linkage or distance matrix

Before any clustering is performed, it is required to determine the proximity matrix containing the distance between each point using a distance function. Then, the matrix is updated to display the distance between each cluster. The following three methods differ in how the distance between each cluster is measured.

**Single Linkage** 		
In single linkage hierarchical clustering, the distance between two clusters is defined as the shortest distance between two points in each cluster. For example, the distance between clusters “r” and “s” to the left is equal to the length of the arrow between their two closest points.
<img src=http://www.saedsayad.com/images/Clustering_single.png>

**Complete Linkage**		
In complete linkage hierarchical clustering, the distance between two clusters is defined as the longest distance between two points in each cluster. For example, the distance between clusters “r” and “s” to the left is equal to the length of the arrow between their two furthest points.
<img src=http://www.saedsayad.com/images/Clustering_complete.png>

**Average Linkage**	
In average linkage hierarchical clustering, the distance between two clusters is defined as the average distance between each point in one cluster to every point in the other cluster. For example, the distance between clusters “r” and “s” to the left is equal to the average length each arrow between connecting the points of one cluster to the other.
<img src=http://www.saedsayad.com/images/Clustering_average.png>


In [ ]:
df = pd.read_csv('./Mall_Customers.csv')
df.head(10)

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(8,5))
plt.title("Annual income distribution",fontsize=16)
plt.xlabel ("Annual income (k$)",fontsize=14)
plt.grid(True)
plt.hist(df['Annual Income (k$)'],color='orange',edgecolor='k')
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
plt.title("Spending Score distribution",fontsize=16)
plt.xlabel ("Spending Score (1-100)",fontsize=14)
plt.grid(True)
plt.hist(df['Spending Score (1-100)'],color='green',edgecolor='k')
plt.show()

### So, is there a definitive correlation between annual income and spending score? - *Apparently not*

In [ ]:
plt.figure(figsize=(8,5))
plt.title("Annual Income and Spending Score correlation",fontsize=18)
plt.xlabel ("Annual Income (k$)",fontsize=14)
plt.ylabel ("Spending Score (1-100)",fontsize=14)
plt.grid(True)
plt.scatter(df['Annual Income (k$)'],df['Spending Score (1-100)'],color='red',edgecolor='k',alpha=0.6, s=100)
plt.show()

### How about correlation between age and spending score? - *Apparently not*

In [ ]:
plt.figure(figsize=(8,5))
plt.title("Age and Spending Score correlation",fontsize=18)
plt.xlabel ("Age",fontsize=14)
plt.ylabel ("Spending Score (1-100)",fontsize=14)
plt.grid(True)
plt.scatter(df['Age'],df['Spending Score (1-100)'],color='blue',edgecolor='k',alpha=0.6, s=100)
plt.show()

## Strategy
** Therefore, we will explore cluserting the customers based on their annual income and spending score to see if there are distinguisbale clusters which the mall can target **

We could use k-means but we don't have any idea about the number of hidden clusters. We will see that hierarchial clustering with dendograms will give us a good insight on the optimal number of clusters.

## Dendograms
[Dendograms](https://en.wikipedia.org/wiki/Dendrogram) are tree diagrams frequently used to illustrate the arrangement of the clusters produced by hierarchical clustering. The clades are arranged according to how similar (or dissimilar) they are. Clades that are close to the same height are similar to each other; clades with different heights are dissimilar — the greater the difference in height, the more dissimilarity. 

In [ ]:
X = df.iloc[:,[3,4]].values

### _Ward_ distance matrix
We will use 'Ward' distance matrix for this dendogram.
$$d(u,v) = \sqrt{\frac{|v|+|s|}{T}d(v,s)^2+ \frac{|v|+|t|}{T}d(v,t)^2- \frac{|v|}{T}d(s,t)^2}$$

where **$u$** is the newly joined cluster consisting of clusters **$s$** and **$t$**, **$v$** is an unused cluster in the forest, **$T=|v|+|s|+|t|$**, and **$|*|$** is the cardinality of its argument. This is also known as the incremental algorithm.

In [ ]:
plt.figure(figsize=(15,6))
plt.title('Dendrogram')
plt.xlabel('Customers')
plt.ylabel('Euclidean distances')
#plt.grid(True)
dendrogram = sch.dendrogram(sch.linkage(X, method = 'ward'))
plt.show()

### Optimal number of clusters

Often, the optimal number of clusters can be found from a Dendogram is a simple manner.
* Look for the longest stretch of vertical line which is not crossed by any ***extended*** horizontal lines (here *extended* means horizontal lines i.e. the cluster dividers are extended infinitely to both directions).
* Now take any point on that stretch of line and draw an imaginary horizontal line.
* Count how many vertical lines this imaginary lines crossed.
* That is likely to be the optimal number of clusters.

**The idea is shown in the following figure. Here the optimal number of clusters could be 5.**

In [ ]:
plt.figure(figsize=(15,6))
plt.title('Dendrogram')
plt.xlabel('Customers')
plt.ylabel('Euclidean distances')
plt.hlines(y=190,xmin=0,xmax=2000,lw=3,linestyles='--')
plt.text(x=900,y=220,s='Horizontal line crossing 5 vertical lines',fontsize=20)
#plt.grid(True)
dendrogram = sch.dendrogram(sch.linkage(X, method = 'ward'))
plt.show()

In [ ]:
from sklearn.cluster import AgglomerativeClustering
hc = AgglomerativeClustering(n_clusters = 5, metric = 'euclidean', linkage = 'ward')
y_hc = hc.fit_predict(X)

### Plot the clusters and label customer types
* _Careful_ - high income but low spenders
* _Standard_ - middle income and middle spenders
* **_Target group_ - middle-to-high income and high spenders (should be targeted by the mall)**
* _Careless_ - low income but high spenders (should be avoided because of possible credit risk)
* _Sensible_ - low income and low spenders

In [ ]:
plt.figure(figsize=(12,7))
plt.scatter(X[y_hc == 0, 0], X[y_hc == 0, 1], s = 100, c = 'red', label = 'Careful')
plt.scatter(X[y_hc == 1, 0], X[y_hc == 1, 1], s = 100, c = 'blue', label = 'Standard')
plt.scatter(X[y_hc == 2, 0], X[y_hc == 2, 1], s = 100, c = 'green', label = 'Target group')
plt.scatter(X[y_hc == 3, 0], X[y_hc == 3, 1], s = 100, c = 'orange', label = 'Careless')
plt.scatter(X[y_hc == 4, 0], X[y_hc == 4, 1], s = 100, c = 'magenta', label = 'Sensible')
plt.title('Clustering of customers',fontsize=20)
plt.xlabel('Annual Income (k$)',fontsize=16)
plt.ylabel('Spending Score (1-100)',fontsize=16)
plt.legend(fontsize=16)
plt.grid(True)
plt.axhspan(ymin=60,ymax=100,xmin=0.4,xmax=0.96,alpha=0.3,color='yellow')
plt.show()

## Verifying the optimal number of clusters by k-means algorithm

Given a set of observations $(x_1, x_2, …, x_n)$, where each observation is a d-dimensional real vector, [**k-means clustering**](https://en.wikipedia.org/wiki/K-means_clustering) aims to partition the *$n$* observations into *$k$* (≤ *$n$*) sets $S = {S_1, S_2, …, S_k}$ so as to minimize the within-cluster sum of squares (WCSS) (i.e. variance). Formally, the objective is to find:

$${\displaystyle {\underset {\mathbf {S} }{\operatorname {arg\,min} }}\sum _{i=1}^{k}\sum _{\mathbf {x} \in S_{i}}\left\|\mathbf {x} -{\boldsymbol {\mu }}_{i}\right\|^{2}={\underset {\mathbf {S} }{\operatorname {arg\,min} }}\sum _{i=1}^{k}|S_{i}|\operatorname {Var} S_{i}}$$

where $\mu_i$ is the mean of points in $S_i$

We run k-means++ model (k-means with carefully initialized centroids) iterating over number of clusters (1 to 15) and plot the ***within-cluster-sum-of-squares (WCSS) matric*** to determine the optimum number of cluster by elbow method

In [ ]:
wcss = []
for i in range(1, 16):
    kmeans = KMeans(n_clusters = i, init = 'k-means++')
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

with plt.style.context(('fivethirtyeight')):
    plt.figure(figsize=(10,6))
    plt.plot(range(1, 16), wcss)
    plt.title('The Elbow Method with k-means++\n',fontsize=25)
    plt.xlabel('Number of clusters')
    plt.xticks(fontsize=20)
    plt.ylabel('WCSS (within-cluster sums of squares)')
    plt.vlines(x=5,ymin=0,ymax=250000,linestyles='--')
    plt.text(x=5.5,y=110000,s='5 clusters seem optimal choice \nfrom the elbow position',
             fontsize=25,fontdict={'family':'Times New Roman'})
    plt.show()

## Part 2:  Autoencoders
Neural networks can be used to implement autoencoders. We will use PyTorch to implement an auto encoder for the MNIST dataset. First, as always we load the dataset and display it.

In [ ]:
plt.rcParams['figure.figsize'] = 15, 10
  
# Initializing the transform for the dataset
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5), (0.5))
])
  
# Downloading the MNIST dataset
train_dataset = torchvision.datasets.MNIST(
    root="./MNIST/train", train=True,
    transform=torchvision.transforms.ToTensor(),
    download=True)
  
test_dataset = torchvision.datasets.MNIST(
    root="./MNIST/test", train=False,
    transform=torchvision.transforms.ToTensor(),
    download=True)
  
# Creating Dataloaders from the
# training and testing dataset
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=1024)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1024)
  
# Printing 25 random images from the training dataset
random_samples = np.random.randint(
    1, len(train_dataset), (25))
  
for idx in range(random_samples.shape[0]):
    plt.subplot(5, 5, idx + 1)
    plt.imshow(train_dataset[idx][0][0].numpy(), cmap='gray')
    plt.title(train_dataset[idx][1])
    plt.axis('off')
  
plt.tight_layout()
plt.show()

Next we define the models for the encoder and decoder parts of the network. Any neural network architecture can be use to implement either part. Since the MNIST is a relatively simple dataset, we will use simple fully connected deep networks, however a convolutional neural network might be preferred for richer images.

In [ ]:
class DeepAutoencoder(nn.Module):
    def __init__(self):
        super().__init__()        
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(28 * 28, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 10)
        )
          
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(10, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 28 * 28),
            torch.nn.Sigmoid()
        )
  
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


model = DeepAutoencoder()
criterion = torch.nn.MSELoss()
num_epochs = 100
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

We now define our training process, which iterates over the entire datasets for 100 epochs. Each step of the loop
iterates over each batch and calculates loss between the predicted image and the original image.
We also store images and their outputs for each epoch.
After the loop ends, we plot out the training loss to better understand the training process. 

In [ ]:
# List that will store the training loss
train_loss = []
  
# Dictionary that will store the
# different images and outputs for 
# various epochs
outputs = {}
  
batch_size = len(train_loader)
  
# Training loop starts
for epoch in range(num_epochs):
        
    # Initializing variable for storing 
    # loss
    running_loss = 0
    stime= time.time()
    # Iterating over the training dataset
    for batch in train_loader:
            
        # Loading image(s) and
        # reshaping it into a 1-d vector
        img, _ = batch  
        img = img.reshape(-1, 28*28)
          
        # Generating output
        out = model(img)
          
        # Calculating loss
        loss = criterion(out, img)
          
        # Updating weights according
        # to the calculated loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
          
        # Incrementing loss
        running_loss += loss.item()
      
    # Averaging out loss over entire batch
    running_loss /= batch_size
    train_loss.append(running_loss)
    print(f"Epoch : {epoch}, loss : {running_loss}, time taken : {time.time()-stime}")
      
    # Storing useful images and
    # reconstructed outputs for the last batch
    outputs[epoch+1] = {'img': img, 'out': out}
  
  
# Plotting the training loss
plt.plot(range(1,num_epochs+1),train_loss)
plt.xlabel("Number of epochs")
plt.ylabel("Training Loss")
plt.show()

In [ ]:
counter = 1
  
epochs_list = [1, 5, 10, 50, 100]
  
# Iterating over specified epochs
for val in epochs_list:
    
      # Extracting recorded information
    temp = outputs[val]['out'].detach().numpy()
    title_text = f"Epoch = {val}"
      
    # Plotting first five images of the last batch
    for idx in range(5):
        plt.subplot(7, 5, counter)
        plt.title(title_text)
        plt.imshow(temp[idx].reshape(28,28), cmap= 'gray')
        plt.axis('off')
          
        # Incrementing the subplot counter
        counter+=1
    
# Iterating over first five
# images of the last batch
for idx in range(5):
      
    # Obtaining image from the dictionary
    val = outputs[10]['img']
      
    # Plotting image
    plt.subplot(7,5,counter)
    plt.imshow(val[idx].reshape(28, 28),
               cmap = 'gray')
    plt.title("Original Image")
    plt.axis('off')
      
    # Incrementing subplot counter
    counter+=1

## Kohonen Self-Organinzing Maps - Using the sklearn_som package
The Self-Organising Map (SOM) is an unsupervised machine learning algorithm introduced by Teuvo Kohonen in the 1980s [1]. As the name suggests, the map organises itself without any instruction from others. It is a brain-inspired model. A different area of the cerebral cortex in our brain is responsible for specific activities. A sensory input like vision, hearing, smell, and taste is mapped to neurons of a corresponding cortex area via synapses in a self-organising way. It is also known that the neurons with similar output are in proximity. SOM is trained through a competitive neural network, a single-layer feed-forward network that resembles these brain mechanisms.
The SOM algorithm can be broken down into the following steps :
1. <b>Initialisation</b> :
   Weights of neurons in the map layer are initialised.

2. <b>Competitive process</b> :
   Select one input sample and search the best matching unit (BMU) among all neurons in n x m grid using distance measures.

3. <b>Cooperative process</b> :
   Find the proximity neurons of BMU by neighbourhood function.

4. <b>Adaptation process</b> :
   Update the BMU and neighbours' weights by shifting the values towards the input pattern.
   If the maximum count of training iteration is reached, exit. If not, increment the iteration count by 1 and repeat the process from 2.
   
Of course, the package abstracts out most of this and allows us to use it like other sklearn modules.


In [ ]:
penguins = load_penguins().dropna()
data = penguins[['bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g']].to_numpy()
penguins['label'] = pd.factorize(penguins['species'])[0] + 1
labels = penguins['label']

print(data.shape, labels.shape)

Now, just like with any classifier right from sklearn, we will have to build an SOM instance and call .fit() on our data to fit the SOM. We already know that there are 3 classes in the Penguins Dataset, so we will use a 3 by 1 structure for our self organizing map, but in practice you may have to try different structures to find what works best for your data. Let's build and fit the som:

In [ ]:
som = SOM(m=3, n=1, dim=data.shape[1], lr=0.1)
som.fit(data, epochs=50)

Note that when building the instance of SOM, we specify m and n to get an m by n matrix of neurons in the self organizing map.
Also similar to sklearn, let's assign each datapoint to a predicted cluster using the .predict() method:

In [ ]:
predictions = som.predict(data)


Lets plot our results

In [ ]:

df = pd.DataFrame(data, columns=['bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g'])
df['ClassLabel'] = labels

# Plot using seaborn pairplot
sns.pairplot(df, hue='ClassLabel')


In [ ]:
df = pd.DataFrame(data, columns=['bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g'])
df['ClassLabel'] = predictions

# Plot using seaborn pairplot
sns.pairplot(df, hue='ClassLabel')